<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lab 3.02: Statistical Modeling and Model Validation

> Authors: Tim Book, Matt Brems

---

## Objective
The goal of this lab is to guide you through the modeling workflow. In this lesson, you will follow all best practices when slicing your data and validating your model. The goal of this lab is not necessarily to build the best model you can, but to build and evaluate a model, and interpret its results.

## Imports

In [1]:
# Import everything you need here.
# You may want to return to this cell to import more things later in the lab.
# DO NOT COPY AND PASTE FROM OUR CLASS SLIDES!
# Muscle memory is important!
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

## Read Data
The `citibike` dataset consists of Citi Bike ridership data for over 224,000 rides in February 2014.

In [2]:
# Read in the citibike data in the data folder in this repository.
citibike = pd.read_csv('./data/citibike_feb2014.csv')

## Explore the data
Use this space to familiarize yourself with the data.

Convince yourself there are no issues with the data. If you find any issues, clean them here.

In [3]:
citibike.head(3)

tripduration            starttime             stoptime  start station id  \
0           382  2014-02-01 00:00:00  2014-02-01 00:06:22               294   
1           372  2014-02-01 00:00:03  2014-02-01 00:06:15               285   
2           591  2014-02-01 00:00:09  2014-02-01 00:10:00               247   

       start station name  start station latitude  start station longitude  \
0     Washington Square E               40.730494               -73.995721   
1      Broadway & E 14 St               40.734546               -73.990741   
2  Perry St & Bleecker St               40.735354               -74.004831   

   end station id          end station name  end station latitude  \
0             265  Stanton St & Chrystie St             40.722293   
1             439            E 4 St & 2 Ave             40.726281   
2             251       Mott St & Prince St             40.723180   

   end station longitude  bikeid    usertype birth year  gender  
0             -73.991475   21101  Subscriber       1991       1  
1             -73.989780   15456  Subscriber       1979       2  
2             -73.994800   16281  Subscriber       1948       2

In [4]:
citibike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224736 entries, 0 to 224735
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tripduration             224736 non-null  int64  
 1   starttime                224736 non-null  object 
 2   stoptime                 224736 non-null  object 
 3   start station id         224736 non-null  int64  
 4   start station name       224736 non-null  object 
 5   start station latitude   224736 non-null  float64
 6   start station longitude  224736 non-null  float64
 7   end station id           224736 non-null  int64  
 8   end station name         224736 non-null  object 
 9   end station latitude     224736 non-null  float64
 10  end station longitude    224736 non-null  float64
 11  bikeid                   224736 non-null  int64  
 12  usertype                 224736 non-null  object 
 13  birth year               224736 non-null  object 
 14  gend

In [5]:
# birth year shouldn't be an object, and starttime / stoptime should be of dtype datetime
citibike[citibike['birth year'] == '\\N']

tripduration            starttime             stoptime  \
31               664  2014-02-01 00:08:47  2014-02-01 00:19:51   
55               836  2014-02-01 00:16:10  2014-02-01 00:30:06   
222             1277  2014-02-01 01:17:50  2014-02-01 01:39:07   
266            29906  2014-02-01 01:44:59  2014-02-01 10:03:25   
293             2625  2014-02-01 01:56:32  2014-02-01 02:40:17   
...              ...                  ...                  ...   
224385           280  2014-02-28 21:42:34  2014-02-28 21:47:14   
224438          1106  2014-02-28 22:00:46  2014-02-28 22:19:12   
224525           864  2014-02-28 22:29:52  2014-02-28 22:44:16   
224536           683  2014-02-28 22:32:55  2014-02-28 22:44:18   
224569           527  2014-02-28 22:44:32  2014-02-28 22:53:19   

        start station id         start station name  start station latitude  \
31                   237            E 11 St & 2 Ave               40.730473   
55                   488            W 39 St & 9 Ave               40.756458   
222                  469         Broadway & W 53 St               40.763441   
266                  294        Washington Square E               40.730494   
293                  395  Bond St & Schermerhorn St               40.688070   
...                  ...                        ...                     ...   
224385               161      LaGuardia Pl & W 3 St               40.729170   
224438               423            W 54 St & 9 Ave               40.765849   
224525               385            E 55 St & 2 Ave               40.757973   
224536               385            E 55 St & 2 Ave               40.757973   
224569               477            W 41 St & 8 Ave               40.756405   

        start station longitude  end station id             end station name  \
31                   -73.986724             349      Rivington St & Ridge St   
55                   -73.993722             297              E 15 St & 3 Ave   
222                  -73.982681             336  Sullivan St & Washington Sq   
266                  -73.995721             368           Carmine St & 6 Ave   
293                  -73.984106             395    Bond St & Schermerhorn St   
...                         ...             ...                          ...   
224385               -73.998102             382      University Pl & E 14 St   
224438               -73.986905             385              E 55 St & 2 Ave   
224525               -73.966033             441              E 52 St & 2 Ave   
224536               -73.966033             441              E 52 St & 2 Ave   
224569               -73.990026             538              W 49 St & 5 Ave   

        end station latitude  end station longitude  bikeid  usertype  \
31                 40.718502             -73.983299   17540  Customer   
55                 40.734232             -73.986923   16731  Customer   
222                40.730477             -73.999061   20728  Customer   
266                40.730386             -74.002150   18944  Customer   
293                40.688070             -73.984106   19782  Customer   
...                      ...                    ...     ...       ...   
224385             40.734927             -73.992005   15529  Customer   
224438             40.757973             -73.966033   18720  Customer   
224525             40.756014             -73.967416   21399  Customer   
224536             40.756014             -73.967416   17516  Customer   
224569             40.757952             -73.977876   20019  Customer   

       birth year  gender  
31             \N       0  
55             \N       0  
222            \N       0  
266            \N       0  
293            \N       0  
...           ...     ...  
224385         \N       0  
224438         \N       0  
224525         \N       0  
224536         \N       0  
224569         \N       0  

[6717 rows x 15 columns]

In [6]:
citibike['gender'].value_counts()

1    176526
2     41479
0      6731
Name: gender, dtype: int64

In [7]:
birth_year_numeric = citibike['birth year'] == "\\N"

In [8]:
birth_year_median = citibike[~birth_year_numeric]['birth year'].median() # Zakaria gave me this idea from class (3.07)

In [9]:
citibike.loc[birth_year_numeric, 'birth year'] = birth_year_median

In [10]:
citibike['birth year'] = citibike['birth year'].apply(lambda x: int(x))

In [11]:
citibike['starttime'] = pd.to_datetime(citibike['starttime']) # converting start and stop time to datetime objects
citibike['stoptime'] = pd.to_datetime(citibike['stoptime'])

In [12]:
citibike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224736 entries, 0 to 224735
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   tripduration             224736 non-null  int64         
 1   starttime                224736 non-null  datetime64[ns]
 2   stoptime                 224736 non-null  datetime64[ns]
 3   start station id         224736 non-null  int64         
 4   start station name       224736 non-null  object        
 5   start station latitude   224736 non-null  float64       
 6   start station longitude  224736 non-null  float64       
 7   end station id           224736 non-null  int64         
 8   end station name         224736 non-null  object        
 9   end station latitude     224736 non-null  float64       
 10  end station longitude    224736 non-null  float64       
 11  bikeid                   224736 non-null  int64         
 12  usertype        

### Is average trip duration different by gender?

Conduct a hypothesis test that checks whether or not the average trip duration is different for `gender=1` and `gender=2`. Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly!

**Hypothesis**: There is no difference in average trip duration between gender 1 and gender 2

In [13]:
# from lesson 3.06. Thanks Patrick Wales-Dinan

gender_1 = citibike[citibike['gender'] == 1]['tripduration']
gender_2 = citibike[citibike['gender'] == 2]['tripduration']

tt = stats.ttest_ind(gender_1, gender_2, equal_var=False)

In [14]:
tt.statistic

-4.802922158264667

In [15]:
tt.pvalue

1.5680482053980446e-06

**Conclusion**: Because our p-value is below our significance level of 0.05, we succeed in rejecting the null hypothesis. We have enough evidence to conclude that the average trip duration differs significantly between the genders.

### What numeric columns shouldn't be treated as numeric?

**Answer:** `Gender`, `start station id`, `end station id`, `bike id`

In [16]:
def get_dummies_and_merge(data, column):
    dummy = pd.get_dummies(data[column], column, drop_first=True) # Create dummies
    return data.merge(dummy, left_index=True, right_index=True) # Merge with original and drop orig column

In [17]:
# creating dummies for gender
citibike = get_dummies_and_merge(citibike, 'gender').drop(columns='gender')

### Dummify the `start station id` Variable

In [18]:
citibike = get_dummies_and_merge(citibike, 'start station id').drop(columns='start station id')

In [19]:
# Create dummies for usertype column
citibike = get_dummies_and_merge(citibike, 'usertype').drop(columns='usertype')

## Feature Engineering
Engineer a feature called `age` that shares how old the person would have been in 2014 (at the time the data was collected)
- Note: you will need to clean the data a bit.

In [20]:
# cleaned data in a previous step, so just need to do the math here!
citibike['age'] = 2014 - citibike['birth year']

In [21]:
# create dummies for the age column
citibike = get_dummies_and_merge(citibike, 'age') # I want to keep this one for later

## Split your data into train/test sets

Look at the size of your data. What is a good proportion for your split? **Justify your answer, considering the size of your data and the default split size in sklearn.**

Use the `tripduration` column as your `y` variable.

For your `X` variables, use `age`, `usertype`, `gender`, and the dummy variables you created from `start station id`. (Hint: You may find the Pandas `.drop()` method helpful here.) 

In [22]:
citibike.shape
citibike.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station name',
       'start station latitude', 'start station longitude', 'end station id',
       'end station name', 'end station latitude', 'end station longitude',
       ...
       'age_88', 'age_92', 'age_93', 'age_97', 'age_101', 'age_104', 'age_107',
       'age_113', 'age_114', 'age_115'],
      dtype='object', length=420)

We could train on 70% of the data and test on the remaining 30%. This would still give us ample data to train on (157,315 observations) and would allow us to ensure our model is strong before putting it into production. 

In [59]:
station_list = [col for col in citibike.columns if 'id_' in col]
gender_list = [col for col in citibike.columns if 'gender_' in col]
usertype_list = [col for col in citibike.columns if 'usertype_' in col]
age_list = [col for col in citibike.columns if 'age_' in col] # will use this later
features = ['age'] + age_list + usertype_list + station_list + gender_list

In [60]:
X = citibike[features]
y = citibike['tripduration']

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

## Fit a Linear Regression model in `sklearn` predicting `tripduration`.

In [62]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

## Evaluate your model
Look at some evaluation metrics for **both** the training and test data. 
- How did your model do? Is it overfit, underfit, or neither?
- Does this model outperform the baseline? (e.g. setting $\hat{y}$ to be the mean of our training `y` values.)

In [63]:
lr.score(X_train, y_train)

0.005688702728836259

In [64]:
lr.score(X_test, y_test)

-2541959244223832.5

In [65]:
mean_squared_error(y_train, lr.predict(X_train)) ** 0.5

5541.065981853022

In [66]:
mean_squared_error(y_test, lr.predict(X_test)) ** 0.5

268080520179.49673

## Fit a Linear Regression model in `statsmodels` predicting `tripduration`.

In [67]:
model = sm.OLS(y, X).fit() 

## Evaluate your model
Using the `statsmodels` summary, test whether or not `age` has a significant effect when predicting `tripduration`.
- Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly **in the context of your model**!

In [70]:
model.summary() # lesson 3.07, thanks Patrick!

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           tripduration   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.480
Date:                Sat, 08 Aug 2020   Prob (F-statistic):           8.40e-53
Time:                        22:22:51   Log-Likelihood:            -2.2534e+06
No. Observations:              224736   AIC:                         4.508e+06
Df Residuals:                  224328   BIC:                         4.512e+06
Df Model:                         407                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
age                     105.0270     24.518      4.284      0.000      56.971     153.083
age_18                 -155.3387    474.880     -0.327      0.744   -1086.091     775.413
age_19                 -343.6593    433.491     -0.793      0.428   -1193.291     505.973
age_20                 -384.4688    439.574     -0.875      0.382   -1246.022     477.084
age_21                 -138.7864    452.370     -0.307      0.759   -1025.421     747.848
age_22                 -555.0086    469.209     -1.183      0.237   -1474.645     364.628
age_23                 -632.6257    486.605     -1.300      0.194   -1586.358     321.107
age_24                 -641.4074    504.123     -1.272      0.203   -1629.475     346.661
age_25                 -836.1234    525.780     -1.590      0.112   -1866.639     194.392
age_26                 -926.9007    547.933     -1.692      0.091   -2000.835     147.034
age_27                -1041.6889    571.080     -1.824      0.068   -2160.992      77.614
age_28                -1089.0960    593.944     -1.834      0.067   -2253.211      75.019
age_29                -1143.0051    616.493     -1.854      0.064   -2351.316      65.306
age_30                -1354.3803    639.788     -2.117      0.034   -2608.349    -100.411
age_31                -1446.1137    663.260     -2.180      0.029   -2746.087    -146.141
age_32                -1463.1517    686.893     -2.130      0.033   -2809.445    -116.858
age_33                -1710.1904    710.440     -2.407      0.016   -3102.635    -317.746
age_34                -1796.5169    734.001     -2.448      0.014   -3235.139    -357.895
age_35                -1854.3106    757.641     -2.447      0.014   -3339.267    -369.354
age_36                -1976.3800    787.966     -2.508      0.012   -3520.774    -431.986
age_37                -1914.6355    805.730     -2.376      0.017   -3493.846    -335.425
age_38                -2127.2668    829.591     -2.564      0.010   -3753.245    -501.289
age_39                -2184.5062    853.860     -2.558      0.011   -3858.050    -510.963
age_40                -2375.1110    877.129     -2.708      0.007   -4094.262    -655.960
age_41                -2403.2026    901.583     -2.666      0.008   -4170.282    -636.123
age_42                -2502.7368    925.277     -2.705      0.007   -4316.257    -689.217
age_43                -2632.0203    949.487     -2.772      0.006   -4492.991    -771.050
age_44                -2821.6910    972.927     -2.900      0.004   -4728.603    -914.778
age_45                -2890.3963    997.027     -2.899      0.004   -4844.544    -936.249
age_46                -3016.1679   1021.428     -2.953      0.003   -5018.141   -1014.195
age_47                -3189.0046   1045.645     -3.050      0.002   -5238.443   -1139.566
age_48                -

**Null Hypothesis:** Age does not affect trip duration

**Alternative Hypothesis:** Age affects trip duration

**Conclusion:** Since our p-value is below our significance level of 0.05, we succeed in rejecting the null hypothesis. We have enough evidence to conclude that `trip duration` differs significantly with changes in `age`.

## Citi Bike is attempting to market to people who they think will ride their bike for a long time. Based on your modeling, what types of individuals should Citi Bike market toward?

In [71]:
coef_df = pd.DataFrame(lr.coef_, X.columns)

In [72]:
coef_df.sort_values(0, ascending=False)

0
age                   5.070621e+05
start station id_353  8.900356e+03
start station id_274  2.939972e+03
start station id_421  2.314782e+03
start station id_343  1.976927e+03
...                            ...
age_107              -4.563574e+07
age_113              -4.867766e+07
age_114              -4.918481e+07
age_115              -4.969269e+07
age_101              -6.960864e+13

[408 rows x 1 columns]

It's very hard to tell Citibike to market to any specific group since there are not many features with statistical significance. Based on the model, the only thing we can say has any significance is that as people get older, they tend to ride bikes for shorter durations. It seems like age 30 would be a good group to market to, that is the first age group that has any statistical significance.

I would ask Citibike for more time to refine my model and not give them bad advice.